# Converting Emulators to Pyomo Expressions

This tutorial's purpose is to walk you through the process of converting your `AutoEmulate` models to Pyomo algebraic expressions for easy optimization workflows. Currently only `PolynomialRegression` and `MLP` are supported for this conversion.

We'll demonstrate the following steps:
1. Training a Neural Network (MLP) on a simple toy function using `AutoEmulate`
2. Exporting the trained network into algebraic expressions compatible with Pyomo
3. Validating the Pyomo expressions against PyTorch predictions

### Prerequisites

* `autoemulate`
* `pyomo`

In [1]:
# General imports for the notebook
import warnings
import numpy as np
import torch
warnings.filterwarnings("ignore")
np.random.seed(42)
torch.manual_seed(42)

## Toy simulation

Before we build an emulator with AutoEmulate, we need to get a set of input/output pairs from our simulation to use as training data.

Below is a simple toy simulation that computes the product of two inputs: `y = x1 * x2`.

In [2]:
# Generate data: y = x1 * x2
n_samples = 1000
x1 = np.random.uniform(-100, 100, size=n_samples)
x2 = np.random.uniform(-100, 100, size=n_samples)

def F(x1, x2):
    return x1 * x2

x = np.column_stack((x1, x2))
y = F(x1, x2)

# Convert to tensors for AutoEmulate
x = torch.tensor(x, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

x.shape, y.shape

(torch.Size([1000, 2]), torch.Size([1000]))

### Data

As you can see, our simulator inputs (`x`) and outputs (`y`) are PyTorch tensors. PyTorch tensors are a common data structure used in machine learning, and `AutoEmulate` is built to work with them.

## Train Emulator

For this tutorial, we'll focus on training an MLP (Multi-Layer Perceptron) neural network.

In [3]:
from autoemulate import AutoEmulate

# Initialize AutoEmulate with MLP model only
ae = AutoEmulate(x, y, log_level="info", models=['MLP'])

INFO    2026-02-05 17:30:02,339 - autoemulate - Comparing ['MLP']
INFO    2026-02-05 17:30:02,340 - autoemulate - Running Model: MLP: 1/1 (attempt 1/3)
INFO    2026-02-05 17:36:00,161 - autoemulate - Finished running Model: MLP

INFO    2026-02-05 17:36:00,163 - autoemulate - Using metric 'r2' to determine best result.


Now that we have run `AutoEmulate`, let's look at the summary for the emulator performance (r-squared and RMSE) on both the train and test data.

In [4]:
ae.summarise()

,model_name,x_transforms,y_transforms,params,r2_test,r2_test_std,rmse_test,rmse_test_std,r2_train,r2_train_std,rmse_train,rmse_train_std
0,MLP,[StandardizeTransform()],[StandardizeTransform()],"{'epochs': 200, 'layer_dims': [32, 16], 'lr': ...",0.99976,0.000034,54.460751,4.148852,0.99978,0.000014,49.899025,1.874318


## Choosing an Emulator

From this list, we can choose an emulator based on the index from the summary dataframe, or quickly get the best performing one using the `best_result` function.

In [5]:
best = ae.best_result()
print("Model with id: ", best.id, " performed best: ", best.model_name)

# best = ae.get_models('MLP')

INFO    2026-02-05 17:36:00,208 - autoemulate - Using metric 'r2' to determine best result.
Model with id:  0  performed best:  MLP


Let's take a look at the configuration of the best model. These are the values of the model's hyperparameters.

In [6]:
print(best.params)

{'epochs': 200, 'layer_dims': [32, 16], 'lr': 0.01, 'batch_size': 16, 'weight_init': 'default', 'scale': 0.1, 'bias_init': 'default', 'dropout_prob': None, 'scheduler_cls': None, 'scheduler_params': {}}


## Converting to Pyomo

Now that we have a trained emulator, we can convert it to Pyomo algebraic expressions. This allows us to use the emulator in optimization problems with mathematical programming solvers.

First, we need to set up a Pyomo model and define decision variables.

In [7]:
import pyomo.environ as pyo

# Create a Pyomo concrete model
pyo_model = pyo.ConcreteModel()

# Pick a test point for initialization
x_init = x[0]

# Define decision variables
# We use the real domain since our training data spans [-100, 100]
pyo_model.x1 = pyo.Var(domain=pyo.Reals)
pyo_model.x2 = pyo.Var(domain=pyo.Reals)

# Initialize to a known point
pyo_model.x1.set_value(x_init[0].item())
pyo_model.x2.set_value(x_init[1].item())

### Export Neural Network to Pyomo

This is the core integration step. The `pyomofy` function converts the neural network's weights, biases, and activation functions into explicit algebraic expressions.

It automatically handles:

* **Input Standardization:** Scaling Pyomo variables to the statistical distribution the network expects
* **Forward Pass:** Generating constraints for every layer
* **Output Inverse Scaling:** Converting the network output back to real-world units
* **ReLU Approximation:** If `nn.ReLU` was used during training, it's automatically approximated with Softplus (controlled by the `relu_beta` parameter). Increase `relu_beta` value if results mismatch significantly, or use a smooth activation function like `SiLU` during training:
```
from torch import nn

# Train with smooth activation for better Pyomo conversion
ae = AutoEmulate(
    x, y, 
    models=['MLP'],
    model_params={'activation_cls': nn.SiLU}  # Use SiLU instead of ReLU
)
```

In [8]:
from autoemulate.convert.pyomo import pyomofy

# Convert the best model to Pyomo expressions
# Pass the Result object (or TransformedEmulator) and the list of Pyomo variables
# Returns a list of expressions (one per output dimension)
emulator_expressions = pyomofy(best, [pyo_model.x1, pyo_model.x2], relu_beta=100)

# Since our output is 1D (the product), we take the first expression
emulator_expr = emulator_expressions[0]

## Verify Numerical Equivalence

Before using the Pyomo expressions in optimization, we **must** verify that they yield the same values as the PyTorch model. This validation ensures the conversion was successful and the algebraic expressions accurately represent the neural network.

In [9]:
# 1. Evaluate Pyomo expression at the initialization point
pyomo_val = pyo.value(emulator_expr)

# 2. Evaluate PyTorch model at the same point
torch_input = x[0].reshape(1, -1)
torch_val = best.model.predict(torch_input).item()

# Print comparison
print(f"Input point:        x1={x_init[0]:.4f}, x2={x_init[1]:.4f}")
print(f"PyTorch prediction: {torch_val:.12f}")
print(f"Pyomo prediction:   {pyomo_val:.12f}")
print(f"\nPyomo vs PyTorch:   {abs(pyomo_val - torch_val):.12f}")

Input point:        x1=-25.0920, x2=-62.9734
PyTorch prediction: 1554.267089843750
Pyomo prediction:   1554.234839389693

Pyomo vs PyTorch:   0.032250454057


## Using the Pyomo Expression

Now that we've validated the conversion, the Pyomo expression can be used in optimization problems. Here's a simple example of how you might set up an objective function:

In [10]:
# Example: Define an objective to maximize the emulator output
pyo_model.obj = pyo.Objective(expr=emulator_expr, sense=pyo.maximize)

# You could also use as constraints, for example:
# pyo_model.constraint1 = pyo.Constraint(expr=emulator_expr >= 1000)